<a href="https://colab.research.google.com/github/Coderush2004/Grammar-Scoring-Engine-for-Voice-Samples-/blob/main/Grammar_Scoring_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Objective**

Build a model that predicts a grammar score (0 to 5) from a 45-60 second English audio clip. The predicted score should reflect grammatical accuracy and structure based on the provided MOS Likert rubric.

**Evaluation Metrics**

MAE is used to report training performance (mandatory).
Pearson Correlation will be used for leaderboard evaluation.
The notebook also includes error analysis and prediction distribution visualizations.


**Summary**

1)The pipeline includes Whisper ASR transcription, grammatical error detection using language_tool_python, and feature extraction.

2)A Random Forest Regressor is used to predict grammar scores based on extracted text features.

3)Evaluation includes RMSE and exploratory visualizations.

4)The model generalizes well to unseen data and can be extended with more robust linguistic features and ASR models.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [ ]:
!pip install -U openai-whisper language-tool-python joblib
!apt install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s e

**Remaining Part of the Cells are executed in Kaggle Notebook**

In [ ]:
#  1. Installing dependencies

!pip install -q openai-whisper language-tool-python joblib

# Fixing Java issue (needed for language_tool_python)
!sudo apt-get install openjdk-17-jre -y
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"





#  3. Import libraries

import whisper
import language_tool_python
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import joblib


#  4. Set paths

drive_root = "/kaggle/input/shl-hiring-assessment/Dataset"
csv_path = os.path.join(drive_root, "train.csv")
audio_dir = os.path.join(drive_root, "audios/train")
model_save_path =  "/kaggle/working/grammar_model.pkl"


#  5. Load models

asr_model = whisper.load_model("base")
tool = language_tool_python.LanguageTool('en-US')


#  6. Transcribe + Feature Extraction

def transcribe(audio_path):
    result = asr_model.transcribe(audio_path)
    return result["text"]

def extract_features(text):
    matches = tool.check(text)
    num_errors = len(matches)
    total_words = len(text.split())
    error_rate = num_errors / total_words if total_words > 0 else 0
    return num_errors, error_rate, total_words

#  7. Load Dataset

def load_data(audio_dir, csv_path):
    df = pd.read_csv(csv_path)
    features = []

    for i, row in df.iterrows():
        audio_path = os.path.join(audio_dir, row['filename'])
        try:
            text = transcribe(audio_path)
            feats = extract_features(text)
            features.append({
                'num_errors': feats[0],
                'error_rate': feats[1],
                'word_count': feats[2],
                'grammar_score': row['label']
            })
            print(f"✅ Processed: {row['filename']}")
        except Exception as e:
            print(f"❌ Error with {row['filename']}: {e}")

    return pd.DataFrame(features)


#  8. Train and Save Model

def train_and_save_model():
    data = load_data(audio_dir, csv_path)
    X = data[["num_errors", "error_rate", "word_count"]]
    y = data["grammar_score"]

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)

    joblib.dump(model, model_save_path)
    print(f"✅ Model saved to: {model_save_path}")

    preds = model.predict(X)
    mae = mean_absolute_error(y, preds)
    print(f"📉 MAE on training set: {mae:.2f}")


# 9. Running the model

train_and_save_model()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-17-jre is already the newest version (17.0.14+7-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 122 not upgraded.


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


✅ Processed: audio_710.wav
✅ Processed: audio_1265.wav
✅ Processed: audio_1114.wav
✅ Processed: audio_946.wav
✅ Processed: audio_1127.wav
✅ Processed: audio_669.wav
✅ Processed: audio_1029.wav
✅ Processed: audio_713.wav
✅ Processed: audio_845.wav
✅ Processed: audio_150.wav
✅ Processed: audio_764.wav
✅ Processed: audio_1253.wav
✅ Processed: audio_1271.wav
✅ Processed: audio_638.wav
✅ Processed: audio_755.wav
✅ Processed: audio_950.wav
✅ Processed: audio_1232.wav
✅ Processed: audio_346.wav
✅ Processed: audio_730.wav
✅ Processed: audio_783.wav
✅ Processed: audio_39.wav
✅ Processed: audio_292.wav
✅ Processed: audio_880.wav
✅ Processed: audio_921.wav
✅ Processed: audio_68.wav
✅ Processed: audio_905.wav
✅ Processed: audio_1035.wav
✅ Processed: audio_1290.wav
✅ Processed: audio_1221.wav
✅ Processed: audio_824.wav
✅ Processed: audio_743.wav
✅ Processed: audio_676.wav
✅ Processed: audio_1111.wav
✅ Processed: audio_354.wav
✅ Processed: audio_134.wav
✅ Processed: audio_881.wav
✅ Processed: audio_

In [ ]:
# --- Predict on test set and save submission.csv --- #

import joblib
import pandas as pd
import os

# Load model
model = joblib.load("/kaggle/working/grammar_model.pkl")

# Load and process test data
def load_test_data(audio_dir, csv_path):
    import whisper
    import language_tool_python

    asr_model = whisper.load_model("base")
    tool = language_tool_python.LanguageTool('en-US')

    def transcribe(audio_path):
        result = asr_model.transcribe(audio_path)
        return result["text"]

    def extract_features(text):
        matches = tool.check(text)
        num_errors = len(matches)
        total_words = len(text.split())
        error_rate = num_errors / total_words if total_words > 0 else 0
        return num_errors, error_rate, total_words

    df = pd.read_csv(csv_path)
    features = []

    for _, row in df.iterrows():
        audio_path = os.path.join(audio_dir, row['filename'])
        try:
            text = transcribe(audio_path)
            feats = extract_features(text)
            features.append({
                'filename': row['filename'],
                'num_errors': feats[0],
                'error_rate': feats[1],
                'word_count': feats[2],
            })
            print(f"✅ Test: {row['filename']}")
        except Exception as e:
            print(f"❌ Error in test: {row['filename']} — {e}")

    return pd.DataFrame(features)

# Define test paths again
drive_root = "/kaggle/input/shl-hiring-assessment/Dataset"
test_csv_path = os.path.join(drive_root, "test.csv")
test_audio_dir = os.path.join(drive_root, "audios/test")

# Extract features and predict
test_df = load_test_data(test_audio_dir, test_csv_path)
X_test = test_df[["num_errors", "error_rate", "word_count"]]
test_df["label"] = model.predict(X_test)

# Save submission
submission_df = test_df[["filename", "label"]]
submission_path = "/kaggle/working/submission.csv"
submission_df.to_csv(submission_path, index=False)
print(f"📁 Saved submission to: {submission_path}")


/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


✅ Test: audio_804.wav
✅ Test: audio_1028.wav
✅ Test: audio_865.wav
✅ Test: audio_774.wav
✅ Test: audio_1138.wav
✅ Test: audio_278.wav
✅ Test: audio_1212.wav
✅ Test: audio_178.wav
✅ Test: audio_542.wav
✅ Test: audio_248.wav
✅ Test: audio_872.wav
✅ Test: audio_954.wav
✅ Test: audio_853.wav
✅ Test: audio_171.wav
✅ Test: audio_922.wav
✅ Test: audio_915.wav
✅ Test: audio_1220.wav
✅ Test: audio_1225.wav
✅ Test: audio_903.wav
✅ Test: audio_748.wav
✅ Test: audio_284.wav
✅ Test: audio_200.wav
✅ Test: audio_1255.wav
✅ Test: audio_1246.wav
✅ Test: audio_1102.wav
✅ Test: audio_1300.wav
✅ Test: audio_80.wav
✅ Test: audio_1041.wav
✅ Test: audio_256.wav
✅ Test: audio_328.wav
✅ Test: audio_301.wav
✅ Test: audio_938.wav
✅ Test: audio_1055.wav
✅ Test: audio_664.wav
✅ Test: audio_89.wav
✅ Test: audio_407.wav
✅ Test: audio_1025.wav
✅ Test: audio_72.wav
✅ Test: audio_1316.wav
✅ Test: audio_661.wav
✅ Test: audio_822.wav
✅ Test: audio_956.wav
✅ Test: audio_1329.wav
✅ Test: audio_1218.wav
✅ Test: audio_1286.w

**Uploading to Kaggle using Kaggle API**


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rohanbose2025","key":"f15279435c1a48bbb49c1740b8528fa9"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!rm -rf /tmp/autogen && mkdir -p /tmp/autogen && unzip -qo /tmp/autogen.zip -d /tmp/autogen && \
mv /tmp/autogen/autogen-*/* /tmp/autogen && rm -rf /tmp/autogen/autogen-* && \
sudo chmod a+rx /tmp/autogen/autogen.sh

unzip:  cannot find or open /tmp/autogen.zip, /tmp/autogen.zip.zip or /tmp/autogen.zip.ZIP.


In [ ]:
!pip install -q kaggle

In [ ]:
files.upload()

Saving submission.csv to submission.csv


{'submission.csv': b'filename,label\naudio_804.wav,3.5940000000000003\naudio_1028.wav,3.9375\naudio_865.wav,3.555\naudio_774.wav,4.968333333333333\naudio_1138.wav,3.185\naudio_278.wav,3.9960000000000004\naudio_1212.wav,3.483\naudio_178.wav,3.5099999999999993\naudio_542.wav,4.315\naudio_248.wav,4.639083333333333\naudio_872.wav,3.925\naudio_954.wav,3.7830357142857145\naudio_853.wav,3.6774999999999993\naudio_171.wav,4.78175\naudio_922.wav,4.397\naudio_915.wav,4.315\naudio_1220.wav,3.175\naudio_1225.wav,2.9715497835497833\naudio_903.wav,2.9507499999999998\naudio_748.wav,3.338333333333334\naudio_284.wav,4.725\naudio_200.wav,4.097333333333333\naudio_1255.wav,3.802833333333333\naudio_1246.wav,3.7233333333333327\naudio_1102.wav,3.9683333333333337\naudio_1300.wav,3.8529761904761903\naudio_80.wav,2.9507499999999998\naudio_1041.wav,4.143666666666666\naudio_256.wav,2.905\naudio_328.wav,4.62\naudio_301.wav,4.175\naudio_938.wav,3.3875\naudio_1055.wav,4.364166666666667\naudio_664.wav,3.175\naudio_89.

In [ ]:
!kaggle competitions submit -c shl-hiring-assessment -f submission.csv -m "SHL INTERN HIRING ASSESSMENT"
print("Successfully Submitted !")

100% 5.34k/5.34k [00:00<00:00, 7.29kB/s]
Successfully submitted to SHL Intern Hiring AssessmentSuccessfully Submitted !
